In [10]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.accountants import RDPAccountant
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager


# Constants
EPOCHS = 3
LOGGING_INTERVAL = 5000
MAX_GRAD_NORM = 1.0
MAX_PHYSICAL_BATCH_SIZE = 8
DELTA = 1e-5
EPSILON = 1.0

In [2]:
# Load dataset
imdb = load_dataset("imdb")

# Tokenizer and Model Initialization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)
imdb = imdb.map(tokenize_function, batched=True)

/home/wentao/anaconda3/envs/shiqi/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define compute metrics function
def compute_metrics(p):
    metric = load_metric("accuracy")
    return metric.compute(predictions=p.predictions.argmax(-1), references=p.label_ids)


In [4]:
# Move the model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

# Instantiate dataloaders
train_dataloader = DataLoader(imdb["train"], shuffle=True, batch_size=32)
test_dataloader = DataLoader(imdb["test"], shuffle=False, batch_size=64)

In [5]:
# Validate the model with Opacus
errors = ModuleValidator.validate(model, strict=True)
if len(errors) == 0:
    print("The model is valid for DP training.")
else:
    print(f"Validation errors: {errors}")
    model = ModuleValidator.fix(model)

The model is valid for DP training.


In [6]:
# Initialize PrivacyEngine
privacy_engine = PrivacyEngine()

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=EPSILON, 
    epochs=EPOCHS,
    max_grad_norm=MAX_GRAD_NORM,
)

/home/wentao/anaconda3/envs/shiqi/lib/python3.8/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/wentao/anaconda3/envs/shiqi/lib/python3.8/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
